# Playground

Having tried many different classification alogorithms without gaining a satisfying accuracy, we are now going to grow the dataset.

Instead of searching for new data from renting sites as "Subito.it" et al. here it is proposed an interactive function to simulate the collection of new ads.

In particular the user will have the opportunity to select new combinations of features and choose wether the target prediction of the algorithm is right or wrong. In the latter case he could pick the true range and store the fresh new informations.

The idea behind is that by doing so the algorithm trained on larger and larger dataset would get more accurate.

## 1. Loading dataset

As done before in "2_RentinBo" we import the dataset and manipulate it to pass it to a classification algorithm, Gaussian Naive Bayes classifier in this case.

In [1]:
import sys
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn import preprocessing
from ipywidgets import interact, interactive, fixed, interact_manual
from IPython.display import display
import ipywidgets as widgets
import functools
ipython = get_ipython()
method_name = "showtraceback"
setattr(
    ipython,
    method_name,
    functools.partial(
        getattr(ipython, method_name),
        exception_only=True
    )
)

In [2]:
dataset = pd.read_csv('play_dataset.csv')
dataset = dataset.drop(columns = 'Unnamed: 0')
dataset

,tipologia,genere,zona,bagno,cucina,salotto,balcone,range euro
0,doppia,F,Murri,2,1,0,0,200-225
1,doppia,F,Marconi,2,1,1,0,225-250
2,doppia,M/F,Bolognina,1,1,1,0,250-275
3,singola,F,S.Vitale,1,1,0,1,400-425
4,singola,F,Irnerio,1,1,0,1,500-525
...,...,...,...,...,...,...,...,...
195,doppia,M,Irnerio,1,1,0,0,200-225
196,doppia,M/F,Bolognina,2,1,1,1,225-250
197,doppia,M/F,Bolognina,1,1,1,1,225-250
198,singola,M/F,San Donato,1,1,0,0,350-375


In [3]:
dummy_dataset = pd.get_dummies(dataset, columns = ['tipologia', 'genere', 'zona'])
columns = list(dummy_dataset.columns.values)
columns.pop(columns.index('range euro')) 
dummy_dataset = dummy_dataset[columns+['range euro']]
dummy_dataset

,bagno,cucina,salotto,balcone,tipologia_doppia,tipologia_singola,genere_F,genere_M,genere_M/F,zona_Barca,...,zona_Malpighi,zona_Marconi,zona_Mazzini,zona_Murri,zona_S.Ruffillo,zona_S.Viola,zona_S.Vitale,zona_Saffi,zona_San Donato,range euro
0,2,1,0,0,1,0,1,0,0,0,...,0,0,0,1,0,0,0,0,0,200-225
1,2,1,1,0,1,0,1,0,0,0,...,0,1,0,0,0,0,0,0,0,225-250
2,1,1,1,0,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,250-275
3,1,1,0,1,0,1,1,0,0,0,...,0,0,0,0,0,0,1,0,0,400-425
4,1,1,0,1,0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,500-525
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,1,1,0,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,200-225
196,2,1,1,1,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,225-250
197,1,1,1,1,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,225-250
198,1,1,0,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,1,350-375


In [4]:
array = dummy_dataset.values
X_enc = array[:,0:27]
Y = array[:,27]

le = preprocessing.LabelEncoder()
le.fit_transform(['175-200', '200-225', '225-250', '250-275', '275-300', '300-325', '325-350', '350-375',
                  '375-400', '400-425', '425-450', '450-475', '475-500', '500-525', '525-550', '550-575', 
                 '575-600', '600-625', '625-650', '650-675', '675-700'])

Y_enc = le.transform(Y)

X_train, X_test, Y_train, Y_test = train_test_split(X_enc, Y_enc, test_size = 0.13, random_state = 843)

GNB_model = GaussianNB(var_smoothing = 1.057)
GNB_model.fit(X_train, Y_train)

GNB_accuracy = GNB_model.score(X_test, Y_test)
print("Accuracy: %.3f%%" % (GNB_accuracy*100.0))

Accuracy: 53.846%


## 2. Let's play

The following function will let the user select all the characteristics of a new ad and then have in response the range of the renting price.

Here will be stored new data.

In [5]:
moredata_X = []
moredata_Y = []

Here are created the interactive widgets.

In [6]:
bagno = widgets.Dropdown(description = 'bagno', options = [0, 1, 2, 3, '-'] , value = '-',
                         continous_update = True)
cucina = widgets.Dropdown(description = 'cucina', options = [0, 1, 2, 3, '-'] , value = '-',
                          continous_update = True)
salotto = widgets.Dropdown(description = 'salotto', options = [0, 1, 2, 3, '-'] , value = '-',
                           continous_update = True)
balcone = widgets.Dropdown(description = 'balcone', options = [0, 1, 2, 3, '-'] , value = '-',
                           continous_update = True)
tipologia = widgets.Dropdown(description = 'tipologia', options = ['singola', 'doppia', '-'], value = '-',
                            continous_update = True)
genere = widgets.Dropdown(description = 'genere', options = ['F', 'M', 'M/F', '-'], value = '-',
                         continous_update = True)
zona = widgets.Dropdown(description = 'zona', options = ['Barca', 'Bolognina', 'Borgo Panigale',
                                                         'Colli', 'Corticella', 'Costa Saragozza',
                                                         'Galvani', 'Irnerio', 'Lame',
                                                         'Malpighi', 'Marconi', 'Mazzini',
                                                         'Murri', 'S.Ruffillo', 'S.Viola',
                                                         'S.Vitale', 'Saffi', 'San Donato', '-'], value = '-',
                       continous_update = True)

y_n = widgets.Dropdown(description = 'Hai scelto?', options = ['Sì', 'No', '-'], value = '-',
                       continous_update = True)
risp = widgets.Dropdown(description = 'È corretto?', options = ['Sì', 'No', '-'], value = '-',
                        continous_update = True)
regi = widgets.Dropdown(description = 'Registra?', options = ['Sì', 'No', '-'], value = '-',
                        continous_update = True)
valu = widgets.Dropdown(description = 'Range', options = ['175-200', '200-225', '225-250', '250-275',
                                                          '275-300', '300-325', '325-350', '350-375',
                                                          '375-400', '400-425', '425-450', '450-475',
                                                          '475-500', '500-525', '525-550', '550-575',
                                                          '575-600', '600-625', '625-650', '650-675',
                                                          '675-700', '-'],
                       value = '-', continous_update = True)
term = widgets.Dropdown(description = 'Terminato?', options = ['Sì', 'No'], value = 'No',
                        continous_update = True)

This is the core of the program, the main function.

In [7]:
def features(bagno, cucina, salotto, balcone, tipologia, genere, zona, y_n, risp, valu, regi, term):
    print('bagno:{} \ncucina:{} \nsalotto:{} \nbalcone:{} \ntipologia:{} \ngenere:{} \nzona:{}'
          .format(bagno, cucina, salotto, balcone, tipologia, genere, zona))
    features = [bagno, cucina, salotto, balcone, tipologia, genere, zona]
    new_X_test = []
    for i in range(0, 7):
        if(i<4):
            new_X_test.append(features[i])
        else:
            if(features[i] == 'singola' or features[i] == 'doppia'):
                if(features[i] == 'singola'):
                    new_X_test.append(0)
                    new_X_test.append(1)
                if(features[i] == 'doppia'):
                    new_X_test.append(1)
                    new_X_test.append(0)
            if(features[i] == 'F' or features[i] == 'M' or features[i] == 'M/F'):
                if(features[i] == 'F'):
                    new_X_test.append(1)
                    new_X_test.append(0)
                    new_X_test.append(0)
                if(features[i] == 'M'):
                    new_X_test.append(0)
                    new_X_test.append(1)
                    new_X_test.append(0)
                if(features[i] == 'M/F'):
                    new_X_test.append(0)
                    new_X_test.append(0)
                    new_X_test.append(1)
            if(features[i] == 'Barca' or features[i] == 'Bolognina' or features[i] == 'Borgo Panigale' or
                features[i] == 'Colli' or features[i] == 'Corticella' or features[i] == 'Costa Saragozza' or
                features[i] == 'Galvani' or features[i] == 'Irnerio' or features[i] == 'Lame' or
                features[i] == 'Malpighi' or features[i] == 'Marconi' or features[i] == 'Mazzini' or
                features[i] == 'Murri' or features[i] == 'S.Ruffillo' or features[i] == 'S.Viola' or
                features[i] == 'S.Vitale' or features[i] == 'Saffi' or features[i] == 'San Donato'):
                if(features[i] == 'Barca'):
                    new_X_test.append(1)
                    for o in range(0,17):
                        new_X_test.append(0)
                if(features[i] == 'Bolognina'):
                    new_X_test.append(0)
                    new_X_test.append(1)
                    for o in range(0,16):
                        new_X_test.append(0)
                if(features[i] == 'Borgo Panigale'):
                    for o in range(0,2):
                        new_X_test.append(0)
                    new_X_test.append(1)
                    for o in range (0,15):
                        new_X_test.append(0)
                if(features[i] == 'Colli'):
                    for o in range(0,3):
                        new_X_test.append(0)
                    new_X_test.append(1)
                    for o in range (0,14):
                        new_X_test.append(0)
                if(features[i] == 'Corticella'):
                    for o in range(0,4):
                        new_X_test.append(0)
                    new_X_test.append(1)
                    for o in range (0,13):
                        new_X_test.append(0)
                if(features[i] == 'Costa Saragozza'):
                    for o in range(0,5):
                        new_X_test.append(0)
                    new_X_test.append(1)
                    for o in range (0,12):
                        new_X_test.append(0)
                if(features[i] == 'Galvani'):
                    for o in range(0,6):
                        new_X_test.append(0)
                    new_X_test.append(1)
                    for o in range (0,11):
                        new_X_test.append(0)
                if(features[i] == 'Irnerio'):
                    for o in range(0,7):
                        new_X_test.append(0)
                    new_X_test.append(1)
                    for o in range (0,10):
                        new_X_test.append(0)
                if(features[i] == 'Lame'):
                    for o in range(0,8):
                        new_X_test.append(0)
                    new_X_test.append(1)
                    for o in range (0,9):
                        new_X_test.append(0)
                if(features[i] == 'Malpighi'):
                    for o in range(0,9):
                        new_X_test.append(0)
                    new_X_test.append(1)
                    for o in range (0,8):
                        new_X_test.append(0)
                if(features[i] == 'Marconi'):
                    for o in range(0,10):
                        new_X_test.append(0)
                    new_X_test.append(1)
                    for o in range (0,7):
                        new_X_test.append(0)
                if(features[i] == 'Mazzini'):
                    for o in range(0,11):
                        new_X_test.append(0)
                    new_X_test.append(1)
                    for o in range (0,6):
                        new_X_test.append(0)
                if(features[i] == 'Murri'):
                    for o in range(0,12):
                        new_X_test.append(0)
                    new_X_test.append(1)
                    for o in range (0,5):
                        new_X_test.append(0)
                if(features[i] == 'S.Ruffillo'):
                    for o in range(0,13):
                        new_X_test.append(0)
                    new_X_test.append(1)
                    for o in range (0,4):
                        new_X_test.append(0)
                if(features[i] == 'S.Viola'):
                    for o in range(0,14):
                        new_X_test.append(0)
                    new_X_test.append(1)
                    for o in range (0,3):
                        new_X_test.append(0)
                if(features[i] == 'S.Vitale'):
                    for o in range(0,15):
                        new_X_test.append(0)
                    new_X_test.append(1)
                    for o in range (0,2):
                        new_X_test.append(0)
                if(features[i] == 'Saffi'):
                    for o in range(0,16):
                        new_X_test.append(0)
                    new_X_test.append(1)
                    new_X_test.append(0)
                if(features[i] == 'San Donato'):
                    for o in range(0,17):
                        new_X_test.append(0)
                    new_X_test.append(1)
    new_X_test = np.transpose(new_X_test)
    if(term != 'Sì'):
        if(bagno != '-' and cucina != '-' and salotto != '-' and balcone != '-' and tipologia != '-' and
           genere != '-' and zona != '-'):
            if(y_n == 'Sì'):
                test_X = []
                test_X.append(new_X_test)
                new_Y_test = le.inverse_transform(GNB_model.predict(test_X))
                print('\n\n\n\n\n\nLa stanza dovrebbe costare:', new_Y_test, 'euro.')
                if(risp == 'Sì'):
                    print('\nGrande!')
                    if(regi == 'Sì'):
                        moredata_X.append(new_X_test)
                        moredata_Y.append(int(GNB_model.predict(test_X)))
                        print('\n\n\nIl nuovo dato è stato inserito.',
                              '\nAdesso setta le 4 scelte su "No" e scegli nuovi parametri.',
                             '\n:)')
                if(risp == 'No'):
                    print('\nInserisci il valore corretto.')
                    print('\n', valu)
                    if(valu != '-' and regi == 'Sì'):
                        moredata_X.append(new_X_test)
                        moredata_Y.append(int(le.transform([valu])))
                        print('\nIl nuovo dato è stato inserito.',
                              '\nAdesso setta le 4 scelte su "-" e scegli nuovi parametri.',
                             '\n:)')
    else:
        print('\n\n\n\n\n\n\n\n\n\n')
        sys.exit('Abbiamo finito.')
    return new_X_test

The following output permits to work on features and immediately get a predicted price.

In [8]:
out = widgets.interactive_output(features, {'bagno': bagno, 'cucina': cucina, 'salotto': salotto,
                                            'balcone': balcone, 'tipologia': tipologia, 'genere': genere,
                                            'zona': zona, 'y_n': y_n, 'risp': risp, 'valu': valu,
                                            'regi': regi, 'term': term, })

widgets.HBox([widgets.VBox([bagno, cucina, salotto, balcone, tipologia, genere, zona, y_n, risp, valu, regi,
                            term]), out])

In [9]:
moredata_X

[]

In [10]:
moredata_Y

[]

When we have finished we stack the new data to the old training set and refit the model.

In [11]:
X_train = np.vstack((X_train, moredata_X))

ValueError: all the input array dimensions for the concatenation axis must match exactly, but along dimension 1, the array at index 0 has size 27 and the array at index 1 has size 0

In [ ]:
Y_train = np.hstack((Y_train, moredata_Y))

In [ ]:
X_train.shape

In [ ]:
Y_train.shape

In [ ]:
GNB_model.fit(X_train, Y_train)